In [1]:
!pip install transformers

## 1. 데이터 로드 및 정제

In [2]:
#!pip install transformers
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

# 데이터프레임으로 변환
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [3]:
# 결합 후 섞기
train_data = pd.concat([train_snli, train_xnli], ignore_index=True)
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [4]:
train_data.head()

,sentence1,sentence2,gold_label
0,바닷가에서 놀고 있는 두 아이,두 아이가 집을 비운다.,entailment
1,페달 사이클을 탄 남자들은 마을에서 밤 외출을 하는 동안 젊은 옷을 입은 소녀들과 ...,금발 남자들이 금발 소녀들과 이야기를 나누고 있다.,neutral
2,머리에 물 튀기는 어린 소년,어린 소년이 탄산음료에 튀었다.,contradiction
3,빨간색과 검은색 줄무늬 셔츠를 입은 남자가 울타리에 그림을 그리고 있다.,카니발에서 사람들의 초상화를 그리는 남자.,contradiction
4,식당 로비에 서거나 앉아 있는 동안 몇몇 사람들이 이야기를 나누고 있다.,식당 앞에 서 있는 사람들.,entailment


In [5]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [6]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [7]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

# 결측값 및 중복 샘플 제거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)

In [8]:
print('훈련용 샘플 개수 :',len(train_data))
print('검증용 샘플 개수 :',len(val_data))
print('테스트용 샘플 개수 :',len(test_data))

훈련용 샘플 개수 : 941814
검증용 샘플 개수 : 2490
테스트용 샘플 개수 : 5010


## 2. GPT의 입력

In [9]:
# 토큰화를 위해 KoGPT-2 토크나이저
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')

print(tokenizer.decode(0))
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<s>
</s>
<usr>
<pad>
<sys>


In [10]:
max_seq_len = 128

sent1 = '모든 섬사람들이 알고 있듯이, 가장 멋진 만과 해변들 중 많은 것들은 보트로만 도달할 수 있다.'
sent2 = '보트는 일부 지역으로 가는 유일한 여행 수단이다.'

print('문장1 :',sent1)
print('문장2 :',sent2)


문장1 : 모든 섬사람들이 알고 있듯이, 가장 멋진 만과 해변들 중 많은 것들은 보트로만 도달할 수 있다.
문장2 : 보트는 일부 지역으로 가는 유일한 여행 수단이다.


In [11]:
bos_token = [tokenizer.bos_token]
eos_token = [tokenizer.eos_token]

# 첫번째 문장의 앞과 뒤에 시작 토큰 <s>과 종료 토큰 </s>으로 감싼다.
sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token

# 두번째 문장의 앞과 뒤에 시작 토큰 <s>과 종료 토큰 </s>으로 감싼다. 그 후 <unused0>를 붙인다.
sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']

# 두 개의 문장을 연달아 이어붙인 후 정수 인코딩을 수행한다.
tokens = sent1_tokens + sent2_tokens
input_id = tokenizer.convert_tokens_to_ids(tokens)
print('정수 인코딩 전:', tokens)
print('정수 인코딩 후:', input_id)

# 최대 길이로 패딩
input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
print('패딩 후:', input_id)

정수 인코딩 전: ['<s>', '▁모든', '▁섬', '사람들이', '▁알고', '▁있듯이,', '▁가장', '▁멋진', '▁만과', '▁해변', '들', '▁중', '▁많은', '▁것들은', '▁보', '트로', '만', '▁도달할', '▁수', '▁있다.', '</s>', '<s>', '▁보', '트는', '▁일부', '▁지역으로', '▁가는', '▁유일한', '▁여행', '▁수단이', '다.', '</s>', '<unused0>']
정수 인코딩 후: [0, 9548, 9709, 34539, 12487, 42370, 9278, 43719, 34766, 23545, 7285, 9044, 9366, 24860, 9049, 11714, 7489, 48699, 9025, 10960, 1, 0, 9049, 11943, 9616, 14303, 11318, 13382, 12079, 26626, 9016, 1, 9]
패딩 후: [    0  9548  9709 34539 12487 42370  9278 43719 34766 23545  7285  9044
  9366 24860  9049 11714  7489 48699  9025 10960     1     0  9049 11943
  9616 14303 11318 13382 12079 26626  9016     1     9     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3 

In [12]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids = []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token
        sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']
        tokens = sent1_tokens + sent2_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids

In [13]:
# 훈련 데이터에 대해서 전처리
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 941814/941814 [06:32<00:00, 2400.57it/s]


In [14]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    0 15217 17157 10624  6889  9080  9174 10607     1     0  9174 25701
 15024  9072  8094  9016     1     9     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : <s> 바닷가에서 놀고 있는 두 아이</s><s> 두 아이가 집을 비운다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [15]:
# 검증 데이터에 대해서 전처리
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 2490/2490 [00:00<00:00, 2931.49it/s]


In [16]:
# 최대 길이: 128
input_id = X_val[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    0  9394  9871  9135  8718 14364 10063  8013 37144  9265 12583  8006
 25856   377     1     0  9258 10192  9848 11001 10644 10396 18796 20485
 37472  9134 35673  9539 18174     1     9     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : <s> 그리고 그가 말했다, "엄마, 저 왔어요."</s><s> 그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [17]:
# 테스트 데이터에 대해서 전처리
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 5010/5010 [00:01<00:00, 2863.12it/s]


In [18]:
# 최대 길이: 128
input_id = X_test[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    0  9621  7925   387 12102 29295 12931  9658 15651 11242 29585 12102
 12371 21837  9938 10305 13076  9427 11120  9124 21016     1     0 12102
 13967  9427 11120  9328 36902     1     9     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : <s> 글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다.</s><s> 나는 그와 다시 이야기하지 않았다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [19]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 고유한 정수로 변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레이블과 정수 :', label_idx)

각 레이블과 정수 : {np.str_('contradiction'): np.int64(0), np.str_('entailment'): np.int64(1), np.str_('neutral'): np.int64(2)}


In [20]:
print('변환 전 :', test_label[:5])
print('변환 후 :',y_test[:5])

변환 전 : ['contradiction', 'entailment', 'neutral', 'neutral', 'entailment']
변환 후 : [0 1 2 2 1]


## 3. GPT의 출력 이해하기

In [21]:
# 모델 로드
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [22]:
# 입력 정의
max_seq_len = 128
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name='input_ids')

# Lambda로 GPT2 모델 감싸기
# outputs : GPT-2의 최종 hidden state
outputs = tf.keras.layers.Lambda(lambda x: model(input_ids=x)[0])(input_ids_layer)

# 마지막 출력 벡터
print(outputs[0])

# 마지막 출력 벡터
print(outputs[0][:, -1])

<KerasTensor shape=(128, 768), dtype=float32, sparse=False, name=keras_tensor_3>
<KerasTensor shape=(128,), dtype=float32, sparse=False, name=keras_tensor_5>


In [ ]:
print(outputs[0][0])

## 4. GPT를 이용한 다중 클래스 분류 모델

In [23]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [24]:
# TPU 작동을 위한 코드
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

# with strategy.scope():
#  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
#  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
#  loss = tf.keras.losses.SparseCategoricalCrossentropy()
#  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
import tensorflow as tf
from transformers import TFGPT2Model
from tensorflow.keras.callbacks import EarlyStopping
import os

# GPT2 기반 분류 모델 정의
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs.last_hidden_state[:, -1]  # outputs[0] → outputs.last_hidden_state
        prediction = self.classifier(cls_token)
        return prediction

# GPU 전략 설정 (선택사항)
# strategy = tf.distribute.MirroredStrategy()

# with strategy.scope():  # GPU에서 굳이 strategy.scope는 필요 없음 (단일 GPU일 경우)
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Early stopping 설정
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)

# 모델 학습
model.fit(
    X_train, y_train,
    epochs=2,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

Epoch 1/2


KeyboardInterrupt: 

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=1024)
print("test loss, test acc: ", results)